In [3]:
import pandas as pd
import re
from collections import OrderedDict

In [4]:
with open('../artforum index v3.txt') as f:
    catalog = f.read()
    
catalog = catalog.replace("NOTE: Please read issue and page references as in the following example:", "") #removing troublesome note lines
catalog = catalog.replace("VI:8, 40-5 April 1968 = Volume VI (6), number 8, pages 40 through 45 April 1968", "")

In [5]:
split_on_index = re.split(r'^([A-Z]{1})$', catalog, flags=re.MULTILINE)

In [6]:
intro = split_on_index[0]
del split_on_index[0]
all_entries = OrderedDict(zip(split_on_index[0::2], split_on_index[1::2]))
print (all_entries.keys())

odict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z'])


In [47]:
writer = pd.ExcelWriter('catalog_tabulated_test.xlsx', engine='xlsxwriter')

for index_letter, content in all_entries.items():
    regex_str = r'^({}[A-Z]*\b.*)$'.format(index_letter)
    entries = re.split(regex_str, content, flags=re.MULTILINE)
    title_data_list = []
    del entries[0]  ## first string returned by split is notes and blank lines
    title = None
    data = None
    for entry in entries:
        if re.match(regex_str, entry):
            title_data_list.append((title, data))
            title = entry.strip()
            parse_entry_title(title)
            data = None
        else:
            data = entry.strip()
            #parse_entry_data(data)
    
    if title_data_list[0][0] is None and title_data_list[0][1] is None:
        del title_data_list[0]
    
    df = pd.DataFrame(title_data_list)
    df.columns = ['entry_title', 'entry_data']
    
    df.to_excel(writer, sheet_name=index_letter)

    
writer.save()

Not a person:  ABSTRACT Expressionism
Not a person:  ABSTRACT Expressionism in California
Not a person:  ACTION Painting
Not a person:  AESOPUS ca. 620-560 B.C.
Not a person:  AFRICA
Not a person:  AIR terminals
Not a person:  AIRCRAFT
Not a person:  ANONYMOUS
Not a person:  ARCHITECTURE
Not a person:  ARCHITECTURE: competitions and awards
Not a person:  ARGENTINA: recent art
Not a person:  ARIZONA: recent art
Not a person:  ARIZONA: reviews
Not a person:  ARMORY Show
Not a person:  ART
Not a person:  ART: surveys
Not a person:  ART and society
Not a person:  ART and technology
Not a person:  ART as investment
Not a person:  ART criticism
Not a person:  ART education
Not a person:  ART history
Not a person:  ART Nouveau
Not a person:  ARTIST and society
Not a person:  ARTISTS on art
Not a person:  ARTS & architecture (periodical)
Not a person:  ARTS magazine
Not a person:  ASIAN art
Not a person:  ASSEMBLAGE
Not a person:  AUTHENTICATION of works of art
Not a person:  AUTOMATIC paintin

In [44]:
#todo

def parse_entry_title(title_string):
    try:
        dates = re.search(r'(\d{4}-.*)$', title_string).group(1)
    except:
        dates = None
        
    if dates is not None:
        title = title_string[:title_string.find(dates)]
        dates = title_string[title_string.find(dates):]
    else:
        title = title_string
    
    if ',' not in title:
        print ('Not a person: ', title)
    return title, dates

def parse_entry_data(data_string):
    datas = data.split('\n')
    for d in datas:
        if 'Exhibition' in d:
            print ("exhibition line found", d)
        else:
            if ':' in d:
                work_type = re.search(r'(.*):(.*)', d).group(1)
                work_title = re.search(r'(.*):(.*)', d).group(2)
                print ("type line found", work_type)
                print ("work title found", work_title)
            else:
                print('unknown line: ', d)